<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/CV/6_Segmentacion/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios Clase 6

En este notebook tendrán que implementar una nueva arquitectura de segmentación semántica llamada U-Net. Esta arquitectura, a diferencia de las Fully Convolutional vistas en clase, está diseñada para retener la información espacial que se pierde durante el downsampling. De esta manera, los feature maps generados durante el upsampling pueden tener un mejor contexto acerca de la posición de los píxeles originales.

# Seccion 1: Dataset

## Ejercicio 1 

Descargar el Dataset Pascal VOC2012 y realizarle image augmentation como se vio en clase. 

Nota: cambiar el crop size por (316,316) para que coincida con los shapes internos de U-Net

In [ ]:
#inserte su código aquí

# Sección 2: Modelo

El modelo que usaremos en este notebook es la U-Net definida en el paper [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf).

![](https://miro.medium.com/max/720/1*YaLdptIoloK184uJQTH1HA.png)

La arquitectura original tomaba como entrada una Imagen (572x572 según el paper) con dimensión de canal "1" para la imagen en escala de grises y generaba un mapa de segmentación de tamaño (388x388) con dimensión de canal equivalente a 2 que era la cantidad de clases que se entrenó para identificar. 

Sin embargo, en este notebook trabajaremos con imágenes a color y clasificaremos cada pixel en una de 21 clases por lo que los canales de entrada y salida deberán ser 3 y 21 respectivamente.

Unet contiene principalmente tres partes:

1. **El camino de contracción**: este es el lado descendente (lado izquierdo) de la "U", ayuda a obtener las features necesarias para la clasificación a medida que va reduciendo la dimensionalidad. .

2. **El camino de expansión**: este es el lado ascendente (lado derecho) de la "U", ayuda a recuperar la dimensionalidad de las imágenes para poder hacer una clasificación a nivel de píxeles.

3. **Conexiones de Salto**: las flechas grises largas desde el lado de contracción hasta el lado de expansión son las conexiones de salto. Estas se utilizan para "retener la información espacial" perdida durante la reducción de resolución de la imagen. De modo que, el mapa de características de la ruta de expansión pueda obtener un mejor contexto de la posición de los píxeles originales. Es decir las mismas features que se generaron al reducir la dimensionalidad son utilizadas para aumentarla.

## Implementación de la Arquitectura

A continuación deberá implementar la arquitectura de U-Net en PyTorch

### Bloque simple de doble convolución
Una imagen de entrada se pasa a través de un par de convolución con tamaño de kernel de 3x3 y una activación de ReLU sobre ella. La dimensión del canal aumenta de “1” a “64”. Luego se pasa a través de otra convolución exactamente igual y otra ReLU, pero esta vez manteniendo la cantidad de canales. Por último, se aplica dropout con probabilidad 0.2

![](https://miro.medium.com/max/640/1*Uan1yYCi3ZO1xrtLohyWzg.png)

Recuerde que, en nuestro caso, los canales entrantes serán 3 porque son imágenes a color.

### Ejercicio 2 




Complete la implementación de la clase SimpleConvolution para que imite el comportamiento del bloque simple de doble convolución

In [ ]:
import torch
import torch.nn as nn

class SimpleConvolution(nn.Module):
    def __init__(self,input_channel,output_channel):
        ## Complete la función init

    def forward(self, x):
        ## Complete la función forward

In [ ]:
#@markdown Test SimpleConvolution
block = SimpleConvolution(1,64)
inp = torch.rand(4,1,572,572)
out = block(inp)
assert out.shape==(4, 64, 568, 568), "El test no fue superado"
print("El test fue superado.")

### Bloque Downsampling de Doble convolución

Este bloque es exactamente igual al anterior sólo que antes de pasar las entradas por las capas convolucionales, se les aplica un max-pooling de kernel $2 \times 2$. 

![](https://miro.medium.com/max/640/1*9zoULdYOeKQsLWQGExhVlQ.png)

Este bloque se aplica 3 veces consecutivas en U-Net con diferentes cantidades de canales.

### Ejercicio 3


Complete la implementación de la clase DownConvolution para que imite el comportamiento del bloque downsampling de doble convolución.

In [ ]:
class DownConvolution(nn.Module):
    def __init__(self,input_channel,output_channel):
        ## Complete la función init

    def forward(self, x):
        ## Complete la función forward


In [ ]:
#@markdown Test DownConvolution
block = DownConvolution(64,128)
inp = torch.rand(4,64,568,568)
out = block(inp)
assert out.shape==(4, 128, 280, 280), "El test no fue superado"
print("El test fue superado.")

### Bloque Upsampling de Doble Convolución

Este bloque es exactamente igual al bloque simple de doble convolución, solo que al final se añade una convolución transpuesta para hacer upsampling.

Esta convolución transpuesta tiene un kernel de 2x2 y con un stride igual a 2. La dimensión del canal de salida en la convolución transpuesta se reduce a la mitad, ya que estaremos concatenando los feature maps provenientes del camino de contracción.

![](https://miro.medium.com/max/640/1*nmfwdmaW5A7_zxI0BcPcGQ.png)

### Ejercicio 4


Complete la implementación de la clase UpConvolution para que imite el comportamiento del bloque upsampling de doble convolución.

In [ ]:
class UpConvolution(nn.Module):
    def __init__(self,input_channel,output_channel):
      ## Complete la función init

    def forward(self, x):
      ## Complete la función forward

In [ ]:
#@markdown Test UpConvolution
block = UpConvolution(512,256)
inp = torch.rand(4,512,104,104)
out = block(inp)
assert out.shape==(4, 128, 200, 200), "El test no fue superado"
print("El test fue superado.")

### Bloque Final

Este bloque es exactamente igual al bloque simple de doble convolución, solo que al final se añade una convolución $1 \times 1$ para mapear los 64 canales que le llegan a la cantidad de clases deseadas.

![](https://miro.medium.com/max/720/1*cqs5XJRsBXS0RAkdIl_wUQ.png)

Recuerda que nuestro dataset tiene 21 clases, no 2 como el paper original.

### Ejercicio 5


Complete la implementación de la clase LastConvolution para que imite el comportamiento del bloque final.

In [ ]:
class LastConvolution(nn.Module):
    def __init__(self,input_channel,output_channel,num_classes):
        ## Complete la función init

    def forward(self, x):
        ## Complete la función forward

In [ ]:
#@markdown Test LastConvolution
block = LastConvolution(128,64,2)
inp = torch.rand(4,128,392,392)
out = block(inp)
assert out.shape==(4, 2, 388, 388), "El test no fue superado"
print("El test fue superado.")

### Conexiones de Salto

Al concatenar los features map del camino de contracción con los del camino de expansión, los primeros deben recortarse para que coincidan con la dimensión de los segundos.

![](https://miro.medium.com/max/720/1*2XyH7YGv7MuJWPycqx7hew.png)

### Ejercicio 6

Implemente la función crop_img para que el source_tensor se recorte a la dimensión del target_tensor.

In [ ]:
def crop_img(source_tensor, target_tensor):
    ## Complete la función 

In [ ]:
#@markdown Test crop_img
src = torch.rand(4,128,280,280)
target = torch.rand(4,256,200,200)
crop_tensor = crop_img(src,target)
assert crop_tensor.shape==(4, 128, 200, 200), "El test no fue superado"
print("El test fue superado.")

### Modelo Completo

Ahora deberá utilizar los bloques generados para ensamblar la U-Net

![](https://miro.medium.com/max/720/1*YaLdptIoloK184uJQTH1HA.png)

### Ejercicio 7

Implemente la clase UNet a partir de los bloques anteriores para que contenga a la red completa.

In [ ]:
class UNet(nn.Module):
    def __init__(self, input_channel, num_classes):
        ## Complete la función init

    def forward(self, x):
        ## Complete la función init

In [ ]:
#@markdown Test UNet
block = UNet(1, 2)
inp = torch.rand(4, 1, 572, 572)
out = block(inp)
print(out.size())
assert out.shape==(4, 2, 388, 388), "El test no fue superado"
print("El test fue superado.")

# Entrenamiento

Nota: debido a las limitaciones de RAM de GPU provistas por Colab, no podrá ejecutar el entrenamiento si ya ha corrido los tests. Los test crean variables que ocupan memoria y se termina acabando y rompiendo el kernel de Colab. Para poder ejecutar el entrenamiento, deberá reiniciar el entorno de ejecución y ejecutar las siguientes celdas sin haber ejecutado los tests.



In [ ]:
def loss(inputs, targets):
    return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)
    
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [ ]:
num_epochs, lr, wd, device = 5, 0.003, 1e-3, torch.device('cuda' if torch.cuda.is_available() else 'cpu')
unet = UNet(3, 21)
trainer = torch.optim.SGD(unet.parameters(), lr=lr, weight_decay=wd)
model = unet.to(device)

In [ ]:
for epoch in range(num_epochs):
    L = 0.0
    N = 0
    Acc = 0.0
    Acc_N = 0
    TestAcc = 0.0
    TestN = 0
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        y = crop_img(y.unsqueeze(1),y_hat)
        y = y.squeeze(1)
        l = loss(y_hat,y)
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
        L += l.sum()
        N += l.numel()
        Acc += accuracy(y_hat,y)
        Acc_N += y.numel()
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        y = crop_img(y.unsqueeze(1),y_hat)
        y = y.squeeze(1)
        TestN += y.numel()
        TestAcc += accuracy(y_hat, y)
    print(f'epoch {epoch + 1}, loss {(L/N):f}\
          , train accuracy  {(Acc/Acc_N):f}, test accuracy {(TestAcc/TestN):f}')